In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import shutil
from sklearn.model_selection import train_test_split

# Define directories for original dataset and split dataset
original_dataset_dir = '/Users/code/alzheimerDetect/axial'
base_dir = '/Users/code/alzheimerDetect/split_dataset_axial'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Define categories
categories = ['AD', 'CN', 'EMCI', 'LMCI']

# Clear old directories if they exist, then create new directories for training and testing data
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

    # List files in the category directory
    category_dir = os.path.join(original_dataset_dir, category)
    files = os.listdir(category_dir)
    
    # Split the data
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    # Copy files to train directory
    for file in train_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(train_dir, category, file)
        shutil.copyfile(src, dst)
    
    # Copy files to test directory
    for file in test_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(test_dir, category, file)
        shutil.copyfile(src, dst)

print("Dataset split into training and testing sets.")

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2  # 20% of the data will be used for validation
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0
)

# Load training data and validation data from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='validation'  # Set as validation data
)

# Load test data from the test directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

# Print the class indices to verify correct classes
print("Class indices:", train_generator.class_indices)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.2f}")

Dataset split into training and testing sets.
Found 2329 images belonging to 4 classes.
Found 581 images belonging to 4 classes.
Found 729 images belonging to 4 classes.
Class indices: {'AD': 0, 'CN': 1, 'EMCI': 2, 'LMCI': 3}


/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-11 13:21:50.714447: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-11 13:21:50.718650: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-11 13:21:50.720652: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-11 13:21:50.726948: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-11 13:21:50.726976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:

Epoch 1/10


2024-11-11 13:21:54.339670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


72/72 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.2693 - loss: nan - val_accuracy: 0.2517 - val_loss: nan
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0938 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/10


2024-11-11 13:23:52.668807: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-11 13:23:52.668838: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/add_14/_52]]
2024-11-11 13:23:52.668865: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7636598271967014755
2024-11-11 13:23:52.668875: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 15162237694967042485
2024-11-11 13:23:52.668878: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16157401480198025417
2024-11-11 13:23:52.668881: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13358348

52/72 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.2413 - loss: nan

2024-11-11 13:25:08.962282: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/Users/code/alzheimerDetect/split_dataset_axial/train/LMCI/Screenshot 2024-08-05 021551.png'
Traceback (most recent call last):

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 260, in _get_iterator
    for i, batch in enumerate(gen_fn()):

  File "/opt/anaconda3/envs/t

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: '/Users/code/alzheimerDetect/split_dataset_axial/train/LMCI/Screenshot 2024-08-05 021551.png'
Traceback (most recent call last):

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/anaconda3/envs/tf_env/lib/python3.9/si